In [3]:
import os
import requests
from bs4 import BeautifulSoup

# Read HTML content from file
html_file = 'data/LTO code.html'
with open(html_file, 'r', encoding='utf-8') as f:
    html_content = f.read()

# Parse HTML to extract PDF links and titles
soup = BeautifulSoup(html_content, 'html.parser')
pdf_links = []

for link in soup.find_all('a', href=True):
    href = link['href']
    if href.endswith('.pdf'):
        title = link.text.strip() or os.path.basename(href)
        pdf_links.append((href, title))

In [4]:
len(pdf_links)

1377

In [7]:
# Directory to save PDFs
output_dir = "data/downloaded_pdfs"
os.makedirs(output_dir, exist_ok=True)

In [13]:
import time

In [36]:
response = requests.get("https://lto.gov.ph/robots.txt")
print(response.text)

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>Microsoft-Azure-Application-Gateway/v2</center>
</body>
</html>



In [1]:
import http.client

In [9]:
def download_and_extract_metadata(pdf_links):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Referer': 'https://lto.gov.ph/',
        'Origin': 'https://lto.gov.ph/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Pragma': 'no-cache'
    }

    for url, title in pdf_links:
        pdf_path = os.path.join(output_dir, f"{title.replace('/', '_')}.pdf")
        try:
            connection = http.client.HTTPSConnection("lto.gov.ph")
            connection.request("GET", url.replace("https://lto.gov.ph", ""), headers=headers)
            response = connection.getresponse()

            if response.status == 200:
                with open(pdf_path, 'wb') as f:
                    f.write(response.read())
                print(f"Downloaded: {title}.pdf")

                # Extract metadata
                with open(pdf_path, 'rb') as f:
                    reader = PdfReader(f)
                    metadata = reader.metadata
                    
                    print(f"Metadata for {title}.pdf:")
                    for key, value in metadata.items():
                        print(f"  {key}: {value}")
                    print("\n")
            else:
                print(f"Failed to download {title}.pdf - Status Code: {response.status}")
        except Exception as e:
            print(f"Error downloading {title}.pdf: {str(e)}")
        finally:
            connection.close()

In [5]:
pdf_links[1][0]

'https://lto.gov.ph/wp-content/uploads/2024/12/Memo11272024.pdf'

In [10]:
# Run the function
download_and_extract_metadata([pdf_links[0]])


Failed to download Nationwide implementation of the online document submission facility through the LTO-IT System.pdf - Status Code: 403
